In [1]:
import os
import pickle
import pystan
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import multiprocessing
import itertools as it
from scipy.special import expit

In [2]:
multiprocessing.set_start_method("fork")
sns.set()  # Nice plot aesthetic

In [3]:
bkp_folder = 'bkp'
os.makedirs(bkp_folder, exist_ok=True)

## Model

## Generate data

In [4]:
np.random.seed(123)

In [5]:
def u(x, alpha):
    return x**(1 - alpha)

In [6]:
# steps = np.linspace(0.01,0.99, 10)
# task = pd.DataFrame(np.array(list(it.product(steps, repeat=4))), columns=["p0", "x0", "p1", "x1"])
task = pd.DataFrame(np.random.random(size=(1000, 4)), columns=["p0", "x0", "p1", "x1"])
task = task[~((task.p0 >= task.p1) & (task.x0 >= task.x1))]
task = task[~((task.p1 >= task.p0) & (task.x1 >= task.x0))]
task.reset_index(inplace=True, drop=True)

In [7]:
n_trial = len(task)

tau = 0.1
true_alpha = 0.4

seu0 = task.p0 * u(task.x0, true_alpha)
seu1 = task.p1 * u(task.x1, true_alpha)

p_choice_1 = scipy.stats.norm.cdf(seu1 - seu0) # expit((seu1 - seu0)/tau)

choice = np.zeros(n_trial, dtype=int)
choice[:] = p_choice_1 > np.random.random(size=n_trial)
task["choice"] = choice
task

,p0,x0,p1,x1,choice
0,0.696469,0.286139,0.226851,0.551315,1
1,0.480932,0.392118,0.343178,0.729050,0
2,0.438572,0.059678,0.398044,0.737995,0
3,0.634401,0.849432,0.724455,0.611024,1
4,0.426351,0.893389,0.944160,0.501837,0
...,...,...,...,...,...
500,0.116790,0.934562,0.280638,0.189423,0
501,0.924761,0.156253,0.298057,0.178112,1
502,0.167810,0.716638,0.535513,0.600545,1
503,0.521893,0.741486,0.875329,0.448498,0


In [8]:
n = 500 
selec = np.random.choice(np.arange(len(task)), size=n, replace=False)
p = np.concatenate((task.p0.values[selec], task.p1.values[selec])).T
x = np.concatenate((task.x0.values[selec], task.x1.values[selec])).T
y = task.choice.values[selec]
# print("p", p)
# print("x", x)
# print("y", y)
data = {"p": p, "x": x, "y": y, "n_x": len(x), "n_y": len(y), "tau": tau}

### Compile the model

In [9]:
model = """
data {
    int<lower=1> n_y;
    int y[n_y];
    int<lower=1> n_x;
    real x[n_x];
    real p[n_x];
    real tau;
}

transformed data {
}

parameters {
    real<lower=0> rho_u;
    real<lower=0> alpha_u;
    real<lower=0> rho_mu;
    real<lower=0> alpha_mu;
    vector[n_x] u;
    
    // vector[n_x] mu_u;
}

transformed parameters {
    matrix[n_x, n_x] K_u = cov_exp_quad(x, alpha_u, rho_u) + diag_matrix(rep_vector(1e-9, n_x));
    vector[n_x] mu_u = rep_vector(0, n_x);
    
    // matrix[n_x, n_x] K_mu = cov_exp_quad(x, alpha_mu, rho_mu) + diag_matrix(rep_vector(1e-9, n_x));
    // vector[n_x] mu_mu = rep_vector(0, n_x);
}

model {    
    vector[2] v;
    vector[2] p_choice;
    int c;
    real p_c;
    
    rho_u ~ normal(0, 3);
    alpha_u ~ normal(0, 1);
    
    //rho_mu ~ normal(0, 3);
    // alpha_mu ~ normal(0, 1);
    // mu_u ~ multi_normal(mu_mu, K_mu);
    
    u ~ multi_normal(mu_u, K_u);
        
    for (i in 1: n_y) {
        v[1] = p[i] * u[i];
        v[2] = p[i+n_y] * u[i+n_y];
        p_choice = softmax(v ./ tau);
        c = y[i]+1;
        p_c = p_choice[c];
        target += log(p_c);
    }
  
}
generated quantities {
}
"""

In [10]:
# Put it to true if you edit the model
force_compilation = True

# Where to save backup
bkp_folder = 'bkp'
os.makedirs(bkp_folder, exist_ok=True)
bkp_file = os.path.join(bkp_folder, 'gp_utility_naive.pkl')

if not os.path.exists(bkp_file) or force_compilation is True:
    
    # Compile the model
    sm = pystan.StanModel(model_code=model)
    
    # Save the model
    with open(bkp_file, 'wb') as f:
        pickle.dump(sm, f)
else:
    # Load the model
    sm = pickle.load(open(bkp_file, 'rb'))

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_26ab5d73ef8803b35d95d89a000127ed NOW.


### Sampling

In [ ]:
# Put it to true if you want to do a new run
force_run = True

# Where to save backup
bkp_file = os.path.join(bkp_folder, 'gp_utility_naive_fit.pkl')

if not os.path.exists(bkp_file) or force_run is True:
    
    # Train the model and generate samples
    fit_stan = sm.sampling(data=data, 
                           iter=1000, chains=1, ) #algorithm="Fixed_param")
    
    # Save
    with open(bkp_file, 'wb') as f:
        pickle.dump(fit_stan, f)
else:
    # Load
    fit_stan = pickle.load(open(bkp_file, 'rb'))

fit_stan

In [ ]:
ex = fit_stan.extract()
len(data["x"])
len(np.mean(ex["u"], axis=0))

x_ = np.linspace(0, 1, 100)

# Create fig
fig, ax = plt.subplots(figsize=(6, 6))

# Set limits
# x_min, x_max = 0, 1
# y_min, y_max = 0, 1

# Plot
ax.plot(x_, u(x_, true_alpha), label="True")

# Pimp your plot
ax.set_xlabel('x')
ax.set_ylabel('u(x)')

# ax.set_xlim(x_min, x_max)
# ax.set_ylim(y_min, y_max)

coord = np.vstack((data["x"], np.mean(ex["u"], axis=0)))
ax.scatter(coord[0], coord[1], alpha=0.5, color='C1')

plt.show()